In [456]:
import pandas as pd

In [480]:
data = pd.read_csv('./sql_interview_questions.csv', sep=';')

## Data Cleaning
Removing duplicate basis question and answer columns since they are the key columns 

In [481]:
data = data.dropna(subset=['question', 'answer'])
print(f"Count of (rows, columns): {data.shape}")
data = data[data['example_query'] != '"example_query"']
print(f"count of unique questions: {data['question'].nunique()}, max id column number: {data['id'].max()}")
data.set_index('id', inplace=True)
data.columns

Count of (rows, columns): (602, 10)
count of unique questions: 601, max id column number: 610


Index(['question', 'answer', 'category', 'difficulty_level', 'tags',
       'example_query', 'explanation', 'common_mistakes', 'related_questions'],
      dtype='object')

1. clean index that is column id contain values over 602 but data only has 602 rows this means id column has wrong values and require deletion of existing index and regenrate the index column named as id.
2. empty values in category they are not marked as na or empty cell so need to find and remove them
3. empty values in difficulty level they are not marked as na or empty cell so need to find and remove them
4. empty values in example_query they are not marked as na or empty cell so need to find and remove them
5. empty values in explanation they are not marked as na or empty cell so need to find and remove them
6. empty values in common_mistakes they are not marked as na or empty cell so need to find and remove them
7. not sure about related_questions column need to check if there are empty values like "" or []

In [482]:
if data.index.max() > 602:
    data.reset_index(drop=True, inplace=True)
print(data.index.max(), data.shape)

600 (601, 9)


In [483]:
# data = data.map(lambda x: x.strip('\r\n').strip('".""').strip('"').strip(' "').strip('" ') if isinstance(x, str) else x)

data = data.map(lambda x: x.strip(' \r\n."') if isinstance(x, str) else x)

# print(data['category'].unique())
data.replace({'category':{'':'SQL','unknown':'SQL'}}, inplace=True)


valid_level = ['easy', 'intermediate', 'advanced']


def clean_difficulty_level(value):
    if value.lower() in valid_level:
        return value.lower()
    return 'unknown' # None


data['difficulty_level'] = data['difficulty_level'].apply(clean_difficulty_level)
# print(data['difficulty_level'].unique())

In [484]:
length = data['example_query'].apply(lambda x: len(x) if isinstance(x, str) else 0)
print(length.max(), length.mean(), length.min())
print(len(data[(length >= length.min()) & (length < length.mean())]['example_query']))

data.replace({'example_query': {'Intermediate': 'Not Available'}}, inplace=True)
data.replace({'example_query': {'': 'Not Available'}}, inplace=True)
data.fillna({'example_query': 'Not Available'}, inplace=True)
print(data[(length < 20)]['example_query'].unique())

461 66.9783693843594 0
436
['Not Available']


In [485]:
# for few rows moving column data
print( (data[data['explanation'] == 'example_query']).head(1))
# print(data.columns, data.columns[ 3:])
data.loc[data['explanation'] == 'example_query', data.columns[2:-1]] = data.loc[data['explanation'] == 'example_query', data.columns[3:]].values
# data.loc[mask, data.columns[2:-1]] = data.loc[mask, data.columns[3:]].value
print('\n\n',data.loc[71])
# print( (data[data['explanation'] == 'example_query']).head(1))

                                             question  \
71  You are given two tables: 'employees' (employe...   

                                               answer category  \
71  SELECT d.department_name FROM employees e JOIN...      SQL   

   difficulty_level          tags                               example_query  \
71          unknown  Intermediate  ['Join', 'Aggregate function', 'Group By']   

      explanation                                    common_mistakes  \
71  example_query  The query joins the 'employees' and 'departmen...   

                                    related_questions  
71  Candidates might forget to use the HAVING clau...  


 question             You are given two tables: 'employees' (employe...
answer               SELECT d.department_name FROM employees e JOIN...
category                                                       unknown
difficulty_level                                          Intermediate
tags                        ['Join', 'Aggrega

In [486]:

length = data['explanation'].apply(lambda x: len(x) if isinstance(x, str) else 0)
print(length.max(), length.mean(), length.min())
print(len(data[(length >= length.min()) & (length < length.mean())]['explanation']))

data.replace({'explanation': {'N/A': 'Not Available', '': 'Not Available'}}, inplace=True)
# data.fillna({'example_query': 'Not Available'}, inplace=True)
print(data[(length < 20)]['explanation'].unique())


524 155.74875207986688 0
245
['Not Available' 'Join, Subquery']


In [488]:
# data.columns
length = data['common_mistakes'].apply(lambda x: len(x) if isinstance(x, str) else 0)

data.replace({'':'Not Available'}, inplace = True)

print(data[lengths < 20]['common_mistakes'].unique())

length = data['related_questions'].apply(lambda x: len(x) if isinstance(x, str) else 0)

data.replace({'':'Not Available'}, inplace = True)

print('\n\n',data[lengths < 25]['related_questions'].unique())

data.to_csv('')
# data.loc[71:76]

['SELECT c.customer_name, SUM(o.total_amount) AS total_spent, COUNT(o.order_id) AS number_of_orders FROM customers c JOIN orders o ON c.customer_id = o.customer_id GROUP BY c.customer_name ORDER BY total_spent DESC LIMIT 5'
 'SELECT category, product_name, total_sales FROM ranked_sales WHERE rank <= 3'
 'SELECT e.name AS employee_name, d.department_name, (SELECT e2.name FROM employees AS e2 WHERE e2.id = d.manager_id) AS manager_name FROM employees AS e JOIN departments AS d ON e.department_id = d.id'
 'WITH CitySales AS ( SELECT c.city, SUM(s.amount) AS total_sales FROM sales s JOIN customers c ON s.customer_id = c.customer_id GROUP BY c.city ) SELECT city, total_sales, RANK() OVER (ORDER BY total_sales DESC) AS rank FROM CitySales'
 'CREATE TABLE salespeople (salesperson_id SERIAL PRIMARY KEY, name VARCHAR(100), hire_date DATE)'
 'WITH EmployeeProjects AS (SELECT e.name AS employee_name, p.project_name, COALESCE(SUM(p.hours_logged), 0) AS total_hours FROM employees e LEFT JOIN projec

In [489]:
!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py

--2024-10-06 13:04:01--  https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3832 (3.7K) [text/plain]
Saving to: ‘minsearch.py’

minsearch.py        100%[===================>]   3.74K  --.-KB/s    in 0.001s  

2024-10-06 13:04:02 (6.37 MB/s) - ‘minsearch.py’ saved [3832/3832]



In [490]:
import minsearch 

# Initialize the search engine
search_engine = MinSearch()

# Assuming you want to index both 'question' and 'answer' fields
for idx, row in data.iterrows():
    search_engine.add_document(str(row['id']), row['question'] + " " + row['answer'])

# Save the index
search_engine.save_index('index_file')


ImportError: cannot import name 'MinSearch' from 'minsearch' (/home/brahm/LLM ZOOMCAMP/local_parent_folder/sql_prep_assistant/notebooks/minsearch.py)